In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import csv
#import pandasql as ps
import matplotlib.pyplot as plt
#import shapefile as shp
import seaborn as sns
import random
import time
from sklearn.model_selection import train_test_split
#import pysal as ps
import scipy

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data

from torch.nn import functional
from torch.autograd import Variable

import statistics

#temporal pattern clustering
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from tslearn.utils import to_time_series_dataset
from sklearn import metrics

#!pip install dtaidistance
#!pip install plotly
#!pip install tslearn
#source https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis

from scipy.cluster.hierarchy import fcluster
from scipy.stats import chisquare
from scipy.stats import wasserstein_distance

#regression model
#from patsy import dmatrices
#import statsmodels.api as sm

from datetime import datetime

#from pysal.model import spreg
#from pysal.lib import weights
#from pysal.explore import esda

#calculate the demographic bias 
#use wasserstein_distance
from scipy.stats import wasserstein_distance

#path = 'C:/Users/29700/Downloads/RecSys-Workshop/tutorials/'
#os.chdir(path)
#import utilities as utl

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mae
from datetime import datetime
import time
from sklearn.decomposition import NMF

import time
import scipy.sparse
from scipy.stats import entropy
from sys import exit

import itertools
from itertools import permutations

import math

import ast

from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mae

from numpy.linalg import svd

In [18]:
pd.read_csv(home_path, header=None, sep='\t')

,0,1,2
0,38343,47.596215,-121.979614
1,13824,47.836323,-122.296098
2,18910,47.222268,-122.493760
3,15894,47.613766,-122.342089
4,5157,47.313734,-122.424222
...,...,...,...
110875,453437,47.330805,-122.170224
110876,455318,47.322828,-122.282397
110877,450532,47.158069,-122.462895
110878,454334,47.944066,-122.224009


In [8]:
#function
#read latlon id with home identified
def read_data(filepath, homepath, combine_dt):
    #read ids with home identified
    id_withhome = pd.read_csv(homepath, header=None, sep='\t')
    id_withhome.columns = ['newid','lat','lon']
    if combine_dt == True:
        id_latlon = pd.read_csv(filepath, header=0, sep=',')
        #get ids
        homeids = list(np.unique(id_withhome.newid))
        id_latlon = id_latlon[id_latlon.newid.isin(homeids)]
        return(id_latlon)
    else:
        return(id_withhome)
    #print(id_latlon.head(3))
    
def convert_to_shp(filepath, homepath, combine_dt=True):
    points = read_data(filepath, homepath, combine_dt)
    geo_points = gpd.GeoDataFrame(points,
                                 geometry=gpd.points_from_xy(points['lon'],\
                                                           points['lat']),\
                                crs={'init': 'epsg:4326'})
    return(geo_points)

#make sure the default crs for zone file is espg4326
def spatial_join(zone_path, points):
    Zone = gpd.read_file(zone_path)
    Zone.crs = points.crs
    points = gpd.tools.sjoin(points, Zone, how='right')
    return(points)

#conduct a small test to make sure the spatial join works
def small_sjoin_test(zonepath, data, key_col):
    test_dt = spatial_join(zonepath, data.iloc[:15])
    geoid = np.unique(test_dt[key_col])
    if 'nan' in geoid:
        print('join not successfully')
        print(geoid)
    else:
        print(geoid)
    return(test_dt)
    
#plot zone and points to make sure they are in the same projection
def plot_point_zone(zone_path, points):
    zone = gpd.read_file(zone_path)
    base = zone.plot(color='white', edgecolor='grey')
    points.plot(ax=base, marker='o', color='red', markersize=5)
    
#check null data after spatial join
def checknull(data_shape, zone_path, savepath, save_index=True):
    data_shape['geoid'] = data_shape['GEOID10'].tolist()
    null_Data = data_shape[data_shape['geoid'].isnull()]
    print(len(null_Data['geoid']))
    plot_point_zone(zone_path, null_Data)
    if save_index == True:
        null_Data.to_csv(savepath)

## Spatial join process

In [ ]:
#PREVIOUS CODE
paths = 'G:/My Drive/2021/Bias/PSRC_od_simulation_2018/'
input_path = paths+'taz2010_revised.shp'
taz = gpd.read_file(input_path)

home_path = 'G:/My Drive/2020/Bias/data-processing/'+'cuebiq_home_psrc_raw.csv'
home_shp = convert_to_shp('G:/', home_path, combine_dt=False)
home_withtaz = spatial_join(input_path, home_shp)

home_withtaz.to_csv(paths+'home_psrc_withtaz.csv')

#get Seattle home ids
os.chdir('G:/My Drive/2020/Bias/data-processing/')
home_withct = spatial_join('spatial_check/Seattle_ct2010.shp', home_shp)
Seattle_homeids = sorted(home_withct['newid'].values)

#latlon_shape_ct = latlon_shape_ct[~latlon_shape_ct['GEOID10'].isnull()]
#latlon_shape_ct.shape

In [24]:
#based on sumo simulation scale
selected_taz = [426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438,
                439, 440, 441, 442, 443, 446, 447, 448, 450, 451, 452, 453, 454,
                455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 
                468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 479, 480, 481, 
                482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 
                495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 
                508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 
                521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533,
                534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546,
                547, 548, 549, 550, 551, 552, 553, 566, 567, 568, 569, 570, 571,
                572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584,
                604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 627,
                628, 629, 630, 631, 632, 634, 635, 636, 650, 651, 652, 653]

In [26]:
home_withtaz.head(3)

,index_left,newid,lat,lon,TAZ,COUNTY_FIP,COUNTY_NM,Area,geometry
0,3480.0,11698.0,47.720013,-122.372924,1,033,King,7117998.77,"POLYGON ((-122.36094 47.72734, -122.36095 47.7..."
0,3643.0,24151.0,47.723837,-122.362050,1,033,King,7117998.77,"POLYGON ((-122.36094 47.72734, -122.36095 47.7..."
0,14927.0,54927.0,47.728959,-122.362530,1,033,King,7117998.77,"POLYGON ((-122.36094 47.72734, -122.36095 47.7..."


In [29]:
#get_data('withobs04052017.csv','cuebiq_home_psrc_raw.csv','spatial_check/ct2010.shp')

def get_new_hr_minut(hr, minut):
    min_index = math.ceil(minut/5)
    if min_index != 12:
        return(hr, min_index*5)
    else: 
        if hr+1 != 24:
            return(hr+1, 0)
        else: 
            return(0, 0)
    
def get_total_locts(obs_data):
    locts = {}
    num_index = 0
    for geoid in np.unique(obs_data['GEOID10']):
        for hr in range(24):
            for minut in np.unique(obs_data['min_5']):
                a = str(geoid)+'_'+str(hr)+'_'+str(minut)
                locts[a] = num_index
                num_index += 1
    nb_locts = len(locts)
    return(locts, nb_locts)

def get_data(obs_data_file, home_file, zone_file, newids, newid_dict):
    latlon_shape = convert_to_shp(obs_data_file, home_file)
    print('original data shape', latlon_shape.shape)
    latlon_shape = spatial_join(zone_file, latlon_shape)
    print('data shape after data join', latlon_shape.shape)
    latlon_shape = latlon_shape[~latlon_shape['GEOID10'].isnull()]
    print('data shape after deleting nulls',latlon_shape.shape)
    
    #summary the obs
    #revise the data, revised as 5-min 
    new_hr = []
    minut_5 = []
    for hr, minut in latlon_shape[['hr', 'minut']].values:
        h, min_5 = get_new_hr_minut(hr, minut)
        new_hr.append(h)
        minut_5.append(min_5)
    
    latlon_shape['newh'] = new_hr
    latlon_shape['min_5'] = minut_5

    group_col = ['newid','GEOID10', 'd', 'newh', 'min_5']
    latlon_shape = latlon_shape[['newid','GEOID10', 'd', 'newh', 'min_5', 'obs']].groupby(by=group_col).sum()
    latlon_shape.reset_index(inplace=True)
    latlon_shape.sort_values(by=group_col)
    
    cur_newids = list(np.unique(latlon_shape['newid']))
    add_ids = []
    for ids in cur_newids:
        if ids not in newids:
            newids.append(ids)
            add_ids.append(ids)
    n_newid = len(newids)
    
    if add_ids != []:
        cur_index = len(newid_dict)
        for ids in add_ids:
            newid_dict[ids] = cur_index 
            cur_index += 1
    
    stids, n_stid = get_total_locts(latlon_shape)
    latlon_shape['stids'] = latlon_shape['GEOID10']+'_'+latlon_shape['newh'].astype(str)+'_'+latlon_shape['min_5'].astype(str)
    
    obs_matrix = np.zeros((n_newid, n_stid))
    for ids, stid, obs in latlon_shape[['newid', 'stids', 'obs']].values:
        obs_matrix[newid_dict[ids], stids[stid]] = obs
    
    print('generate obs matrix shape', obs_matrix.shape)
    path = 'G:/My Drive/2021/Bias/SUMO_simulation/'
    savefile = 'obsmx'+obs_data_file[7:-4]+'.csv'
    pd.DataFrame(obs_matrix).to_csv(path+savefile)
    
    return(newids, newid_dict)

def get_total_locts_taz(obs_data):
    locts = {}
    num_index = 0
    for geoid in np.unique(obs_data['TAZ']):
        for hr in range(24):
            for minut in np.unique(obs_data['min_5']):
                a = str(geoid)+'_'+str(hr)+'_'+str(minut)
                locts[a] = num_index
                num_index += 1
    nb_locts = len(locts)
    return(locts, nb_locts)

def get_data_taz(obs_data_file, home_file, zone_file, newids, newid_dict):
    latlon_shape = convert_to_shp(obs_data_file, home_file)
    print('original data shape', latlon_shape.shape)
    latlon_shape = spatial_join(zone_file, latlon_shape)
    print('data shape after data join', latlon_shape.shape)
    latlon_shape = latlon_shape[~latlon_shape['TAZ'].isnull()]
    print('data shape after deleting nulls',latlon_shape.shape)
    
    #summary the obs
    #revise the data, revised as 5-min 
    new_hr = []
    minut_5 = []
    for hr, minut in latlon_shape[['hr', 'minut']].values:
        h, min_5 = get_new_hr_minut(hr, minut)
        new_hr.append(h)
        minut_5.append(min_5)
    
    latlon_shape['newh'] = new_hr
    latlon_shape['min_5'] = minut_5

    group_col = ['newid','TAZ', 'd', 'newh', 'min_5']
    latlon_shape = latlon_shape[['newid','TAZ', 'd', 'newh', 'min_5', 'obs']].groupby(by=group_col).sum()
    latlon_shape.reset_index(inplace=True)
    latlon_shape.sort_values(by=group_col)
    
    cur_newids = list(np.unique(latlon_shape['newid']))
    add_ids = []
    for ids in cur_newids:
        if ids not in newids:
            newids.append(ids)
            add_ids.append(ids)
    n_newid = len(newids)
    
    if add_ids != []:
        cur_index = len(newid_dict)
        for ids in add_ids:
            newid_dict[ids] = cur_index 
            cur_index += 1
    
    stids, n_stid = get_total_locts_taz(latlon_shape)
    latlon_shape['stids'] = latlon_shape['TAZ']+'_'+latlon_shape['newh'].astype(str)+'_'+latlon_shape['min_5'].astype(str)
    
    obs_matrix = np.zeros((n_newid, n_stid))
    for ids, stid, obs in latlon_shape[['newid', 'stids', 'obs']].values:
        obs_matrix[newid_dict[ids], stids[stid]] = obs
    
    print('generate obs matrix shape', obs_matrix.shape)
    path = 'G:/My Drive/2021/Bias/SUMO_simulation/'
    savefile = 'obstazmx'+obs_data_file[7:-4]+'.csv'
    pd.DataFrame(obs_matrix).to_csv(path+savefile)
    
    return(newids, newid_dict)

'G:/My Drive/2021/Bias/PSRC_od_simulation_2018/taz2010_revised.shp'

In [33]:
#get_data_taz('withobs04052017.csv','cuebiq_home_psrc_raw.csv',input_path)
latlon_shape = convert_to_shp('withobs04052017.csv', 'cuebiq_home_psrc_raw.csv')

C:\Users\Yiran\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
#def get_data_taz(obs_data_file, home_file, zone_file, newids, newid_dict):

print('original data shape', latlon_shape.shape)
latlon_shape = spatial_join(zone_file, latlon_shape)
print('data shape after data join', latlon_shape.shape)
latlon_shape = latlon_shape[~latlon_shape['TAZ'].isnull()]
print('data shape after deleting nulls',latlon_shape.shape)

#summary the obs
#revise the data, revised as 5-min 
new_hr = []
minut_5 = []
for hr, minut in latlon_shape[['hr', 'minut']].values:
    h, min_5 = get_new_hr_minut(hr, minut)
    new_hr.append(h)
    minut_5.append(min_5)

latlon_shape['newh'] = new_hr
latlon_shape['min_5'] = minut_5

group_col = ['newid','TAZ', 'd', 'newh', 'min_5']
latlon_shape = latlon_shape[['newid','TAZ', 'd', 'newh', 'min_5', 'obs']].groupby(by=group_col).sum()
latlon_shape.reset_index(inplace=True)
latlon_shape.sort_values(by=group_col)

cur_newids = list(np.unique(latlon_shape['newid']))
add_ids = []
for ids in cur_newids:
    if ids not in newids:
        newids.append(ids)
        add_ids.append(ids)
n_newid = len(newids)

if add_ids != []:
    cur_index = len(newid_dict)
    for ids in add_ids:
        newid_dict[ids] = cur_index 
        cur_index += 1

stids, n_stid = get_total_locts_taz(latlon_shape)
latlon_shape['stids'] = latlon_shape['TAZ']+'_'+latlon_shape['newh'].astype(str)+'_'+latlon_shape['min_5'].astype(str)

obs_matrix = np.zeros((n_newid, n_stid))
for ids, stid, obs in latlon_shape[['newid', 'stids', 'obs']].values:
    obs_matrix[newid_dict[ids], stids[stid]] = obs

print('generate obs matrix shape', obs_matrix.shape)
path = 'G:/My Drive/2021/Bias/SUMO_simulation/'
savefile = 'obstazmx'+obs_data_file[7:-4]+'.csv'
pd.DataFrame(obs_matrix).to_csv(path+savefile)

return(newids, newid_dict)

In [ ]:
#get cuebiq data file name ('withobs04052017.csv')
obs_csv_file = []
for i in os.listdir():
    if 'withobs' in i:
        obs_csv_file.append(i)

#GET THE DATA
newids = []
newid_dict = {}
for obs_file in obs_csv_file:
    newids, newid_dict = get_data(obs_file, 'cuebiq_home_psrc_raw.csv', 'spatial_check/Seattle_ct2010.shp', newids, newid_dict) 
    
with open('newid_tsid_aggregation.txt', 'w') as f:
    f.write(str(newid_dict))
    f.write(str(stids))
f.close()

#n_user = len(newids)
def get_agg_matrix(input_path, n_user, obs_matrix, first_input=True):
    cur_obs_matrix = pd.read_csv(input_path, index_col=0).to_numpy()
    
    cur_n_user, n_sid = cur_obs_matrix.shape
    #print('current obs matrix shape', cur_obs_matrix.shape)
        
    if cur_n_user != n_user:
        diff_n_user = int(n_user-cur_n_user)
        filled_zero_matrix = np.zeros((diff_n_user, n_sid))
        cur_obs_matrix = np.vstack([cur_obs_matrix, filled_zero_matrix])
        #print('revised matrix shape', cur_obs_matrix.shape)
    if first_input == False:
        obs_matrix_revised = obs_matrix + cur_obs_matrix
    else:
        obs_matrix_revised = cur_obs_matrix
    return(obs_matrix_revised)

#get_sum_matrix 
input_path = 'G:/My Drive/2021/Bias/SUMO_simulation/' 
matrixfile = []
for i in os.listdir(input_path):
    if 'obsmx' in i:
        matrixfile.append(i)
        
obs_mx = 0
for mxfile in matrixfile:
    filepath = input_path + mxfile
    if obs_mx == 0:
        obs_matrix_sum = get_agg_matrix(filepath, n_user, obs_mx, True)
        obs_mx = 1
        #print('matrixfile cur obs matrix shape', obs_matrix_sum.shape)
    else:
        obs_matrix_sum = get_agg_matrix(filepath, n_user, obs_matrix_sum, False)
        #obs_mx = obs_matrix.copy()
        #print('matrixfile cur obs matrix shape', obs_matrix_sum.shape)